# Dimensionality reduction

The feature selection and extraction process often leaves us with a very large featureset: each document may be characterized by thousands of token frequencies, for example. In order to effectively look for patterns within these data and visualize the results, we need meaningful ways to prune or combine features.

If this is a new R session, we’ll need to re-load the tidyverse packages.

In [1]:
library(tidyverse)
library(tidytext)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Topic Modeling

Topic modeling is the process of identifying constellations of tokens that tend to co-occur among the documents. This can have two benefits. First, the number of such token groups, or **topics**, is much smaller than the number of tokens. Characterizing the documents in terms of, say, 20 topics is simpler than trying to represent them in terms of thousands of word frequencies. The second potential benefit is that the topics themselves can often be treated as meaningful. Understanding which words contribute most to each topic may grant some insight into the connections between ideas within the documents.

A number of methods can be used for topic modeling, finding the one that works best for your dataset generally involves some experimentation. It is almost always necessary to tune the model based on some test data in order to get the best performance. Today we’ll try some simple experiments with **Latent Dirichlet Allocation** LDA, a technique popularized in the 2000s by David Blei. Specifically, we’ll use the version implemented by the `topicmodels` package.

The results are not guaranteed to be identical every time you run the algorithm, even when you use the same parameters. The model is probabilistic, and the computer builds it by an iterative trial and error process that contains some randomness. You can explicitly set a random number "seed" if you want to make sure that the same set of pseudo-random numbers is used every time. But it’s also helpful to do several runs with different seeds and indeed with slightly different starting parameters and confirm that the results are comparable.

## Choosing a feature set and number of topics

LDA can take a long time. If you are able to fine-tune your feature set to remove very common and very rare terms first, the process will be more efficient and accurate. Unfortunately, determining the number of very common and very rare terms to prune is often itself a trial and error process.

Likewise for ***k***, the number of topics. It is common to re-run the process with many different values of k in order to optimize for the most stable results. Some packages even provide a script to help automate this process.

## Additional datasets

For this example, we can use the **sotu** package, which provides the texts of the annual "State of the Union" address by the President of the United States. I’m also going to use the pre-made English stoplist from the package **NLP** and the English stemmer from **SnowballC**.

In [2]:
library(topicmodels)
library(NLP)
library(SnowballC)
library(sotu)


Attaching package: ‘NLP’


The following object is masked from ‘package:ggplot2’:

    annotate




The **sotu** package provides the texts of the "State of the Union" addresses as a character vector called `sotu_text`. It also provides additional information about each speech, including the year, and the name and party of the President, in a tibble called `sotu_meta`. I’m going to add an `id` column to both so I can correlate them later.

In [3]:
head(sotu_meta)

president,year,years_active,party,sotu_type
<chr>,<int>,<chr>,<chr>,<chr>
George Washington,1790,1789-1793,Nonpartisan,speech
George Washington,1790,1789-1793,Nonpartisan,speech
George Washington,1791,1789-1793,Nonpartisan,speech
George Washington,1792,1789-1793,Nonpartisan,speech
George Washington,1793,1793-1797,Nonpartisan,speech
George Washington,1794,1793-1797,Nonpartisan,speech


In [4]:
sotu_meta <- sotu_meta %>% 
    mutate(id = row_number()) %>%
    relocate(id, .before = president)

In [5]:
head(sotu_meta)

id,president,year,years_active,party,sotu_type
<int>,<chr>,<int>,<chr>,<chr>,<chr>
1,George Washington,1790,1789-1793,Nonpartisan,speech
2,George Washington,1790,1789-1793,Nonpartisan,speech
3,George Washington,1791,1789-1793,Nonpartisan,speech
4,George Washington,1792,1789-1793,Nonpartisan,speech
5,George Washington,1793,1793-1797,Nonpartisan,speech
6,George Washington,1794,1793-1797,Nonpartisan,speech


In [6]:
str(sotu_text)

 chr [1:236] "Fellow-Citizens of the Senate and House of Representatives: \n\nI embrace with great satisfaction the opportuni"| __truncated__ ...


In [7]:
sotu_text <- tibble(text=sotu_text) %>%
    mutate(id = row_number()) %>%
    relocate(id, .before=text)

In [8]:
print(sotu_text)

# A tibble: 236 × 2
      id text                                                                   
   <int> <chr>                                                                  
 1     1 "Fellow-Citizens of the Senate and House of Representatives: \n\nI emb…
 2     2 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 3     3 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 4     4 "Fellow-Citizens of the Senate and House of Representatives: \n\nIt is…
 5     5 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 6     6 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 7     7 "\n\nFellow-Citizens of the Senate and House of Representatives: \n\nI…
 8     8 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 9     9 "\n\n Gentlemen of the Senate and Gentlemen of the House of Representa…
10    10 "\n\n Gentlemen of the Senate and Gentlemen of the House of Representa…
# … with

### Tokenization

The first step is feature extraction. Let’s begin by tokenzing into words and stemming. I'm using SnowballC’s `wordStem()` for stemming. This is a quick-and-dirty method for getting at the underlying "stems" of words that for the most part just chops off anything that looks like an inflectional ending, with no sensitivity to context.

In [9]:
tokens <- sotu_text %>% 
    unnest_tokens(word, text) %>%
    mutate(stem = wordStem(word))

In [10]:
tokens %>% head(20)

id,word,stem
<int>,<chr>,<chr>
1,fellow,fellow
1,citizens,citizen
1,of,of
1,the,the
1,senate,senat
1,and,and
1,house,hous
1,of,of
1,representatives,repres


Let’s calculate corpus-wide stem counts. Then we can make a list of overly-common stems to be omitted from the topic model.

In [11]:
corpus_stem_counts <- tokens %>% count(stem, sort=TRUE)

In [12]:
corpus_stem_counts %>% head(25)

stem,n
<chr>,<int>
the,164556
of,105824
to,67358
and,67193
a,44263
in,43013
i,29211
that,23845
it,22386


Just for now, I’m going to make a MFW list of everything up to 'state' and exclude those terms. In a more careful study I would want to hand-select stopwords here.

In [13]:
stop_mfw <- corpus_stem_counts$stem[1:24]
print(stop_mfw)

 [1] "the"   "of"    "to"    "and"   "a"     "in"    "i"     "that"  "it"   
[10] "be"    "for"   "our"   "by"    "have"  "on"    "thi"   "which" "we"   
[19] "with"  "will"  "ha"    "ar"    "been"  "not"  


To generate a list of *infrequent* words we want to exclude, let’s calculate per-document word counts, and then determine how many documents each word occurs in.

In [14]:
stem_doc_counts <- tokens %>% 
    count(id, stem) %>%
    count(stem, sort=TRUE)

In [15]:
stem_doc_counts

stem,n
<chr>,<int>
a,236
all,236
an,236
and,236
ar,236
at,236
be,236
been,236
but,236


Let’s make a list of words that occur in five documents or fewer.

In [16]:
stop_lfw <- with(stem_doc_counts, stem[n<=4])

In [17]:
length(stop_lfw)

[1] 12823

In [18]:
head(stop_lfw, 100)

[1] "1,200"       "1.5m"        "103"         "107"         "110,000"    
  [6] "11m"         "125"         "140"         "140,000,000" "1753"       
 [11] "180"         "180,000"     "1806"        "1807"        "1843"       
 [16] "18m"         "19,000,000"  "1923"        "1952"        "1956"       
 [21] "196"         "1965"        "1966"        "1967"        "1986"       
 [26] "1994"        "1997"        "1998"        "2.2"         "2.4"        
 [31] "2.6"         "2007"        "2014"        "22,000"      "271"        
 [36] "290"         "3,100"       "3,500,000"   "3.2"         "3.3"        
 [41] "3.65"        "32,000,000"  "33,000,000"  "346,681,016" "38,000"     
 [46] "400,000,000" "425,000"     "450"         "46"          "47,000,000" 
 [51] "4m"          "5,500,000"   "504"         "54,000,000"  "60,000,000" 
 [56] "64"          "650"         "675"         "700,000,000" "71"         
 [61] "77"          "8.5"         "8.5m"        "80th"        "88"         
 [66] "93"          "95th"        "96"          "abhor"       "abomin"     
 [71] "acid"        "activist"    "adhes"       "admiralti"   "affront"    
 [76] "aforesaid"   "aimless"     "alfr"        "aluminum"    "amen"       
 [81] "american'"   "amphitrit"   "angl"        "anguish"     "anthracit"  
 [86] "antonio"     "antwerp"     "apollo"      "appendix"    "arabian"    
 [91] "aristocraci" "aristocrat"  "army'"       "asean"       "astronaut"  
 [96] "astronomi"   "asund"       "attaind"     "auction"     "auguri"

Let’s apply these stoplists to our token table and generate a new set of counts.

In [19]:
token_counts_no_stops <- tokens %>% 
    filter(! stem %in% stop_mfw) %>%
    filter(! stem %in% stop_lfw) %>%
    count(id, stem, sort=TRUE)

In [20]:
print(token_counts_no_stops)

# A tibble: 308,810 × 3
      id stem         n
   <int> <chr>    <int>
 1   199 year       247
 2   158 year       227
 3   119 should     212
 4   158 dollar     212
 5   199 my         205
 6   199 congress   204
 7   158 war        202
 8   199 program    200
 9   201 year       192
10   122 govern     185
# … with 308,800 more rows


### Convert to document-term matrix

In order to use `topicmodels`, we need to convert this tibble, which has one row for each document-term combination, to an enormous matrix that has one row for every document and one column for every stem. Note that most of the values in this matrix will be zero, since most of the stems don’t occur in most of the documents. That mans that this is a "sparse matrix", and R has special ways of dealing with these.

In particular, a number of language-processing packages share a common DTM structure, and tidytext knows how to convert our data to this format. The conversion function is called `cast_dtm()`

In [21]:
dtm <- token_counts_no_stops %>% cast_dtm(id, stem, n)

What are the dimensions of the resulting matrix?

In [22]:
dim(dtm)

[1]  236 6950

So we have 236 documents, for which we’ve measured 6950 stem counts. Using `print()` on the DTM will tell us a little more about its properties, if we’re interested.

In [23]:
print(dtm)

<<DocumentTermMatrix (documents: 236, terms: 6950)>>
Non-/sparse entries: 308810/1331390
Sparsity           : 81%
Maximal term length: 15
Weighting          : term frequency (tf)


## Building a model

Now we can go ahead and run the LDA algorithm to build a topic model. In the interest of time, I’m only going to ask it to generate five topics. That’s probably too few, but with luck it will be illustrative. On my computer this step takes about a minute or two.

In [24]:
lda_model <- LDA(dtm, k=5)

The resulting model is a complex R object, with lots of component attributes. In particular, we’re interested in "beta" the probabilities that each stem is associated with each topic, and "gamma", the probabilities that each topic is associated with each document.

Each of these is a matrix:
 - beta has one row for each topic and a column for every term
     - each topic is seen as made up of all the terms in various proportions
 - gamma has one row for each document and a column for every topic
     - each document is seen as made up of all the topics in various proportions
     
One way to get these values out of the model is with the helper function `posterior()`.

In [25]:
posterior(lda_model)

$terms
         year      should       dollar           my    congress         war
1 0.011158787 0.001629569 2.181702e-03 0.0043492902 0.006978257 0.002335592
2 0.001538974 0.004200443 4.717233e-07 0.0023461161 0.004378629 0.004402242
3 0.002433167 0.011101212 3.688167e-04 0.0005856986 0.003214119 0.003178932
4 0.009379395 0.002389208 5.302890e-04 0.0028116959 0.006685374 0.001153989
5 0.008572874 0.003025830 5.703241e-04 0.0025654359 0.003564543 0.002243296
       program      govern          wa       state      nation       mexico
1 9.168623e-03 0.006291729 0.002291383 0.005526942 0.008573243 8.092229e-05
2 1.301057e-33 0.007579274 0.005213935 0.011487202 0.005946878 1.909640e-03
3 2.038502e-13 0.007024512 0.002234474 0.003405210 0.007434671 9.489684e-07
4 7.039328e-31 0.011451434 0.011313381 0.014393945 0.001187614 1.252264e-03
5 1.463906e-03 0.003136948 0.002749266 0.002799855 0.005716860 4.375602e-05
           or       their        feder       legisl      million       propos
1 0.001948768 0.003872065 0.0069111770 0.0040003663 0.0031483678 0.0041724736
2 0.005539977 0.011179442 0.0006043952 0.0009614825 0.0002704614 0.0008604225
3 0.008204913 0.006021623 0.0010393292 0.0026632779 0.0002726305 0.0007037301
4 0.005460007 0.006474936 0.0003498889 0.0014378382 0.0004820755 0.0006633026
5 0.005109081 0.007385200 0.0011083464 0.0003804530 0.0021362531 0.0014475222
     administr      develop      continu         such        from         all
1 0.0042805400 0.0054243061 0.0055372798 0.0016640066 0.004066424 0.004025765
2 0.0003603043 0.0001988891 0.0014495652 0.0042083722 0.009929699 0.007238945
3 0.0009973127 0.0017585294 0.0007792242 0.0061499999 0.005393594 0.006281792
4 0.0009516688 0.0001187625 0.0019891883 0.0047214454 0.008383890 0.005181509
5 0.0003592127 0.0006372580 0.0013203484 0.0003223776 0.005546308 0.008490373
        fiscal   expenditur         work       polici          new        unit
1 2.016359e-03 1.523553e-03 0.0036970401 0.0040698767 0.0058199135 0.003369236
2 2.246584e-05 8.264087e-04 0.0005187707 0.0015513533 0.0016144713 0.005344701
3 1.838539e-04 8.309955e-04 0.0039639968 0.0013238258 0.0009842066 0.001919604
4 2.115117e-03 1.695591e-03 0.0008172878 0.0009738191 0.0023952289 0.009004457
5 1.125100e-04 7.931405e-09 0.0058683042 0.0004599004 0.0059803196 0.001646340
      increas          an       energi        more         but       provid
1 0.005813376 0.005495071 2.934219e-03 0.005000899 0.002586525 0.0040173310
2 0.001075457 0.006601831 2.813542e-04 0.003790630 0.005813288 0.0011732953
3 0.002317713 0.006007230 3.360074e-05 0.003649902 0.006652623 0.0021154175
4 0.003449691 0.006558261 1.315829e-04 0.002215429 0.002780138 0.0016567596
5 0.001091691 0.003466332 1.189299e-03 0.008270465 0.008746841 0.0009890603
          law         need          had          at         gold       econom
1 0.001544268 0.0048244070 0.0007169771 0.004208906 7.125284e-06 5.259226e-03
2 0.002675160 0.0001790905 0.0044212044 0.006776948 1.122491e-09 1.304479e-04
3 0.005263959 0.0032317542 0.0013211740 0.006533296 1.258699e-04 5.178252e-04
4 0.004447614 0.0002902043 0.0019672755 0.008659815 1.453864e-03 5.330743e-05
5 0.001257321 0.0034132419 0.0012872475 0.004734792 2.186698e-05 1.305878e-03
        there         ani         upon     countri          you       depart
1 0.001324446 0.000876175 0.0006046448 0.002518305 4.396152e-08 0.0013123341
2 0.002235076 0.004665010 0.0042528468 0.004900960 2.574727e-03 0.0007725175
3 0.005590748 0.004755477 0.0047694588 0.004095975 2.903699e-04 0.0021329580
4 0.001979691 0.002381930 0.0062606629 0.004692232 2.846861e-04 0.0035064339
5 0.002767789 0.001280678 0.0001315903 0.003170109 7.864035e-03 0.0001502542
      american       these    constitut          so        world        also
1 0.0029898470 0.005836076 0.0002989895 0.001074688 0.0049131916 0.003722378
2 0.0005750836 0.004269569 0.0036143591 0.004731543 0.0007239871 0.001632656
3 0.0029059984 0.004321679 0.0002646521 0.004235117 0.

But a more human-friendly interface is provided by topicmodels itself, in the form of the helper functions `terms()` and `topics()`. These provide, instead of the raw probabilities, a ranking of the top *n* terms per topic, `terms()`, or topics per document, `topics()`.

In [26]:
terms(lda_model, 20)

Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
year,state,should,state,but
program,their,or,govern,american
nation,from,nation,wa,year
congress,govern,govern,year,all
feder,all,but,unit,more
govern,thei,at,at,you
these,at,all,from,peopl
new,them,such,congress,their
increas,an,their,an,can
continu,mai,an,their,must


In [27]:
topics(lda_model, 10)

199,158,119,201,122,110,113,117,67,111,⋯,14,2,21,22,128,1,11,7,12,169
1,1,3,1,3,4,3,3,2,4,⋯,2,2,2,2,3,2,2,2,2,1
5,3,4,5,4,3,2,2,4,3,⋯,4,4,4,4,2,4,4,4,4,5
4,4,2,4,1,2,5,4,3,1,⋯,3,3,3,1,1,3,3,1,1,3
3,2,1,2,2,5,4,5,1,2,⋯,1,5,1,3,4,5,5,5,3,2
2,5,5,3,5,1,1,1,5,5,⋯,5,1,5,5,5,1,1,3,5,4
